In [1]:
import random
import sys

import pandas as pd

from gcdyn import bdms, gpmap, mutators, responses, utils

sys.path.append("../experiments")

import replay

In [2]:
gp_map = gpmap.ConstantGPMap(1.0)

In [3]:
def make_replay_ish_tree(base_seed):
    tree = bdms.TreeNode()
    tree.sequence = replay.naive_sequence
    tree.x = gp_map(tree.sequence)
    birth_rate = 0.35
    time_to_sampling = 20
    for seed in range(1000):
        try:
            tree.evolve(
                time_to_sampling,
                birth_rate=responses.ConstantResponse(birth_rate),
                mutator=mutators.SequencePhenotypeMutator(replay.mutator, gp_map),
                mutation_rate=replay.mutation_response,
                min_survivors=20,
                seed=base_seed+seed,
            )
            return tree
        except bdms.TreeError:
            print("try failed")
            continue

tree = make_replay_ish_tree(0)
print("tip count: ", len(tree))
tree.sample_survivors(n = 60)
tree.prune()

tip count:  509


In [4]:
print(tree)


               /-1754
            /-|
           |  |   /-1828
           |   \-|
           |      \-1251
         /-|
        |  |      /-1193
        |  |   /-|
        |  |  |  |   /-1752
        |  |  |   \-|
        |   \-|      \-1485
        |     |
        |     |   /-1555
        |      \-|
        |         \-1542
        |
        |         /-913
        |      /-|
        |     |   \-1522
        |     |
      /-|     |         /-1340
     |  |     |      /-|
     |  |   /-|     |   \-1459
     |  |  |  |   /-|
     |  |  |  |  |  |      /-517
     |  |  |  |  |  |   /-|
     |  |  |  |  |   \-|   \-1454
     |  |  |   \-|     |
     |  |  |     |      \-1707
     |  |  |     |
     |  |  |     |   /-1204
     |  |  |      \-|
     |  |  |         \-1718
     |  |  |
     |  |  |         /-1260
     |   \-|      /-|
     |     |     |  |   /-758
     |     |     |   \-|
     |     |     |      \-1469
     |     |   /-|
     |     |  |  |   /-900
     |     |  |  |  |
    

In [5]:
tree.total_mutations = 0

for node in tree.iter_descendants(strategy="preorder"):
    node.total_mutations = node.n_mutations + node.up.total_mutations

[leaf.total_mutations for leaf in tree.iter_leaves()]

[8,
 11,
 9,
 12,
 13,
 14,
 10,
 10,
 11,
 12,
 7,
 9,
 8,
 11,
 12,
 11,
 12,
 9,
 6,
 6,
 4,
 6,
 8,
 8,
 5,
 7,
 10,
 11,
 10,
 9,
 6,
 11,
 9,
 9,
 8,
 12,
 8,
 12,
 12,
 11,
 10,
 12,
 12,
 9,
 8,
 9,
 7,
 7,
 6,
 9,
 8,
 10,
 8,
 14,
 13,
 15,
 11,
 13,
 11,
 10]

In [6]:
with open("tree.nwk", "w") as fp:
    fp.write(tree.write()+"\n")

In [7]:
utils.write_leaf_sequences_to_fasta(tree, "seqs.fasta", naive=replay.naive_sequence)